In [1]:
from typing import TypedDict

class BlogState(TypedDict):
    topic: str
    blog: str
    review_feedback: str
    accuracy: float
    iteration: int


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI


c:\Users\SriHarsha\Desktop\POC\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
writer_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7
)


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [5]:
def writer_agent(state: BlogState) -> BlogState:
    prompt = f"""
    Write a high-quality technical blog on the topic:
    "{state['topic']}"

    Constraints:
    - Maximum 500 words
    - Improve based on this feedback:
    {state.get('review_feedback', 'None')}
    """
    blog = writer_llm.invoke(prompt).content
    return {**state,"blog": blog,"iteration":state["iteration"]+1}


In [8]:
from langchain_openai import ChatOpenAI
import os

In [18]:
reviewer_llm = ChatOpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.environ["NVIDIA_API_KEY"],
    model="meta/llama3-8b-instruct",
    temperature=0
)


In [19]:
def reviewer_agent(state: BlogState) -> BlogState:
    prompt = f"""
    Review the following blog for factual and technical accuracy.

    Blog:
    {state['blog']}

    Tasks:
    1. Give improvement feedback
    2. Provide an accuracy score from 0 to 100

    Respond strictly in this format:
    Accuracy: <number>
    Feedback: <text>
    """
    response = reviewer_llm.invoke(prompt).content
    accuracy = float(response.split("Accuracy:")[1].split("\n")[0].strip())
    feedback = response.split("Feedback:")[1].strip()
    return {**state,"accuracy":accuracy,"review_feedback":feedback}


In [20]:
def supervisor_decision(state: BlogState) -> str:
    if state["accuracy"] >= 85:
        return "end"
    return "writer"


In [21]:
from langgraph.graph import StateGraph, END
    

In [22]:
graph = StateGraph(BlogState)
graph.add_node("writer", writer_agent)
graph.add_node("reviewer", reviewer_agent)
graph.set_entry_point("writer")
graph.add_edge("writer", "reviewer")
graph.add_conditional_edges(
    "reviewer",
    supervisor_decision,
    {
        "writer": "writer",
        "end": END
    }
)
blog_graph = graph.compile()


In [23]:
initial_state = {
    "topic": "Retrieval-Augmented Generation using LangGraph",
    "blog": "",
    "review_feedback": "",
    "accuracy": 0.0,
    "iteration": 0
}


In [24]:
final_state = blog_graph.invoke(initial_state)
print(final_state["blog"])
print(final_state["accuracy"])


## Retrieval-Augmented Generation: Supercharging RAG with LangGraph

Retrieval-Augmented Generation (RAG) has become a cornerstone for building robust, factual, and up-to-date Large Language Model (LLM) applications. By grounding LLM responses in external knowledge, RAG mitigates hallucinations and provides domain-specific insights. However, orchestrating complex RAG workflows – involving multiple retrieval steps, conditional logic, and self-correction – can quickly become challenging. Enter LangGraph, a powerful library built on LangChain, designed to build stateful, multi-actor applications with cyclical capabilities.

### Why LangGraph for RAG?

LangGraph excels at defining workflows as graphs, where each "node" performs a specific task (e.g., retrieve, generate, re-evaluate) and "edges" dictate the flow. This graph-based approach offers several advantages for RAG:

*   **Modularity:** Break down complex RAG into manageable, reusable components.
*   **State Management:** Maintain co